## Quantinuum tutorial

This notebook will show you how to use pytket to run circuits on the Quantinuum backends. You will need to install `pytket-quantinuum`.

### Import the necessary packages

In [1]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter as rcj

## Create a circuit

In [2]:
c = Circuit(2).H(0).CX(0, 1).measure_all()

rcj(c)

## Basic emulator usage

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend('H1-1E', provider='microsoft')
compiled_circuit = backend.get_compiled_circuit(c)
handle = backend.process_circuit(compiled_circuit, n_shots=100)
result = backend.get_result(handle)
counts = result.get_counts()
print(counts)

## Using myqos

Quantinuum supports batching, but the pytket interface is a little cumbersome ATOW (you have to add each circuit manually). With Myqos you can pass `attempt_batch=True` to `QuantinuumConfig` and it will automatically batch a set of circuits for you. This means that once it's your turn in the queue, the whole batch will run together. See more details on the [Quantinuum via Myqos page](https://myqos.com/docs/example_notebooks/quantinuum_examples.html#Batching) and the [pytket-quantinuum docs](https://cqcl.github.io/pytket-quantinuum/api/index.html).

In [4]:
from pytket.extensions.myqos import Myqos, MyqosBackend, QuantinuumConfig

myqos = Myqos()
myqos.login()
quantinuum_config = QuantinuumConfig(device_name='H1-1E', user_group='DEFAULT', attempt_batching=True)

# experiment = myqos.new_experiment('h_series_test')
experiment = myqos.get_experiment_by_name('h_series_test')

backend = MyqosBackend(quantinuum_config, experiment)


Started using experiment with name: h_series_test


In [17]:
circuits = [c, c, c]

compiled_circuits = backend.get_compiled_circuits(circuits)
handles = backend.process_circuits(compiled_circuits, n_shots=100)

In [18]:
results = [backend.get_result(h, timeout=None) for h in handles]
counts = [r.get_counts() for r in results]
print(counts)

[Counter({(1, 1): 60, (0, 0): 40}), Counter({(1, 1): 54, (0, 0): 45, (1, 0): 1}), Counter({(0, 0): 50, (1, 1): 49, (1, 0): 1})]


## Cost estimation

The Quantinuum backends have a `.cost` function which returns the HQCs of a circuit. You may need to explicitly provide the `syntax_checker` you need to use.

In [14]:
from pytket.extensions.quantinuum import QuantinuumBackend
emulator_backend = QuantinuumBackend('H1-1E')

In [12]:
emulator_backend.cost(
    emulator_backend.get_compiled_circuit(c), n_shots=100, syntax_checker="H1-1SC"
)

5.66

A more complicated circuit will be more expensive, as will the same circuit with more shots.

In [13]:
emulator_backend.cost(
    emulator_backend.get_compiled_circuit(c), n_shots=5000, syntax_checker="H1-1SC"
)

38.0